## Understanding monad transformers by building one

`MaybeT`는 `Control.Monad.Trans.Maybe`(transformers)에 정의되어 있으나, 직접 구현해보자.

In [39]:
{-# LANGUAGE FlexibleInstances,MultiParamTypeClasses, UndecidableInstances #-}
module MaybeT (
  MaybeT(..),
  runMaybeT
) where

import Control.Monad
import Control.Monad.Trans
import Control.Monad.Writer

newtype MaybeT m a = MaybeT {
    runMaybeT :: m (Maybe a)
}

bindMT :: (Monad m) => MaybeT m a -> (a -> MaybeT m b) -> MaybeT m b
x `bindMT` f = MaybeT $ do
                unwrapped <- runMaybeT x
                case unwrapped of
                    Nothing -> return Nothing
                    Just y -> runMaybeT (f y)

returnMT :: (Monad m) => a -> MaybeT m a
returnMT a = MaybeT $ return (Just a)

failMT :: (Monad m) => t -> MaybeT m a
failMT _ = MaybeT $ return Nothing

instance (Monad m) => Functor (MaybeT m) where
 fmap = liftM

instance (Monad m) => Applicative (MaybeT m) where
 pure = return
 (<*>) = ap
 
instance (Monad m) => Monad (MaybeT m) where
 return = returnMT
 (>>=) = bindMT
 fail = failMT

instance MonadTrans MaybeT where
 lift m = MaybeT (Just `liftM` m)

instance (MonadWriter w m) => MonadWriter w (MaybeT m) where
 tell = lift . tell
 listen m = MaybeT $ do
             (result, log) <- listen (runMaybeT m)
             case result of
                 Nothing -> return Nothing
                 Just value -> return (Just (value, log))
 pass m = MaybeT $ do
             result <- runMaybeT m
             case result of
                 Nothing -> return Nothing
                 Just (value, log) -> pass (return (Just value, log))

`MonadWriter`뿐만 아니라 여러가지 monad들에 대해 instance를 추가할 수 있다. 그런데, `MonadWriter`는 `lift`만으로 쉽게 구현할 수 없다.

```haskell
writer :: (a, w) -> m a Source
```
writer (a,w) embeds a simple writer action.
```haskell
tell :: w -> m () Source
```
tell w is an action that produces the output w.
```haskell
listen :: m a -> m (a, w) Source
```
listen m is an action that executes the action m and adds its output to the value of the computation.
```haskell
pass :: m (a, w -> w) -> m a Source
```
pass m is an action that executes the action m, which returns a value and a function, and returns the value, applying the function to the output.

## Transformer stacking order is important


Monad transformer는 함수 합성과 비슷해서 합성 순서, 즉 결합 순서가 중요하다.

In [40]:
:set -XFlexibleContexts

In [41]:
import Control.Monad.Writer
import MaybeT

In [42]:
problem :: MonadWriter [String] m => m ()
problem = do
    tell ["this is where i fail"]
    fail "oops"

`MonadWriter`를 사용하는 `problem`함수에 아래의 두 타입이 모두 적용될 수 있다.

In [43]:
type A = WriterT [String] Maybe
type B = MaybeT (Writer [String])

In [44]:
a :: A ()
a = problem

b :: B ()
b = problem

그러나 monad 결합 순서에 따라 결과가 다르다.

In [45]:
runWriterT a

Nothing

In [46]:
runWriter $ runMaybeT b

(Nothing,["this is where i fail"])